In [0]:

from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType
     

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/fa1166e8-ce9c-43e6-852f-03c63d084a87/oauth2/token"}


dbutils.fs.mount(
source = "abfss://olympic-data-container@olpymicprojectashutosh.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/olympic",
extra_configs = configs)

In [0]:
%fs
ls mnt/olympic

path,name,size,modificationTime
dbfs:/mnt/olympic/raw-data/,raw-data/,0,1725211411000
dbfs:/mnt/olympic/transformed-data/,transformed-data/,0,1725211421000


In [0]:
spark

In [0]:
athletes = spark.read.format("csv").option("header","true").load("/mnt/olympic/raw-data/athletes.csv")

In [0]:
athletes.show(5)

+-----------------+-------+-------------------+
|       PersonName|Country|         Discipline|
+-----------------+-------+-------------------+
|  AALERUD Katrine| Norway|       Cycling Road|
|      ABAD Nestor|  Spain|Artistic Gymnastics|
|ABAGNALE Giovanni|  Italy|             Rowing|
|   ABALDE Alberto|  Spain|         Basketball|
|    ABALDE Tamara|  Spain|         Basketball|
+-----------------+-------+-------------------+
only showing top 5 rows



In [0]:
coaches = spark.read.format("csv").option("header","true").load("/mnt/olympic/raw-data/coaches.csv")
entriesgender = spark.read.format("csv").option("header","true").load("/mnt/olympic/raw-data/entriesgender.csv")
medals = spark.read.format("csv").option("header","true").load("/mnt/olympic/raw-data/medals.csv")
teams = spark.read.format("csv").option("header","true").load("/mnt/olympic/raw-data/teams.csv")

In [0]:
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
coaches.show(5)

+---------------+-------------+----------+-----+
|           Name|      Country|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| NULL|
|      ABE Junya|        Japan|Volleyball| NULL|
|  ABE Katsuhiko|        Japan|Basketball| NULL|
|   ADAMA Cherif|C�te d'Ivoire|  Football| NULL|
|     AGEBA Yuya|        Japan|Volleyball| NULL|
+---------------+-------------+----------+-----+
only showing top 5 rows



In [0]:
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
entriesgender.show(5)

+-------------------+------+----+-----+
|         Discipline|Female|Male|Total|
+-------------------+------+----+-----+
|     3x3 Basketball|    32|  32|   64|
|            Archery|    64|  64|  128|
|Artistic Gymnastics|    98|  98|  196|
|  Artistic Swimming|   105|   0|  105|
|          Athletics|   969|1072| 2041|
+-------------------+------+----+-----+
only showing top 5 rows



In [0]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [0]:
entriesgender = entriesgender.withColumn('Female',col('Female').cast('int')).withColumn('Male',col('Male').cast('int')).withColumn('Total',col('Total').cast('int'))

In [0]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
medals.show(5)

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
+----+--------------------+----+------+------+-----+-------------+
only showing top 5 rows



In [0]:
medals.printSchema()

root
 |-- Rank: string (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: string (nullable = true)
 |-- Silver: string (nullable = true)
 |-- Bronze: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- Rank by Total: string (nullable = true)



In [0]:
medals = medals.withColumn('Gold',col('Gold').cast('int')).withColumn('Silver',col('Silver').cast('int')).withColumn('Bronze',col('Bronze').cast('int')).withColumn('Total',col('Total').cast('int')).withColumn('Rank by Total',col('Rank by Total').cast('int'))

In [0]:
medals.printSchema()

root
 |-- Rank: string (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
teams.show(5)

+--------+--------------+--------------------+-----+
|TeamName|    Discipline|             Country|Event|
+--------+--------------+--------------------+-----+
| Belgium|3x3 Basketball|             Belgium|  Men|
|   China|3x3 Basketball|People's Republic...|  Men|
|   China|3x3 Basketball|People's Republic...|Women|
|  France|3x3 Basketball|              France|Women|
|   Italy|3x3 Basketball|               Italy|Women|
+--------+--------------+--------------------+-----+
only showing top 5 rows



In [0]:
teams.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
top_gold_countries = medals.orderBy("Gold",ascending = False).select("TeamCountry","Gold","Rank")

In [0]:
athletes.write.mode("overwrite").option("header","true").csv("/mnt/olympic/transformed-data/athletes")

In [0]:
coaches.write.mode("overwrite").option("header","true").csv("/mnt/olympic/transformed-data/coaches")
entriesgender.write.mode("overwrite").option("header","true").csv("/mnt/olympic/transformed-data/entriesgender")
medals.write.mode("overwrite").option("header","true").csv("/mnt/olympic/transformed-data/medals")
teams.write.mode("overwrite").option("header","true").csv("/mnt/olympic/transformed-data/teams")